# Phase B: Model Diagnostics & Advanced Feature Engineering
**Project:** Real Estate Price Prediction (Queretaro, Mexico)
**Objective:** Diagnose the econometric anomalies found in the Baseline BigQuery models and implement feature engineering to resolve multicollinearity and endogeneity issues.

## 1. Baseline Model "Post-Mortem" Analysis
In Phase A, we trained two baseline models using BigQuery ML:
1.  **Model A (Linear):** Target = `price` ($R^2 \approx 0.645$)
2.  **Model B (Log-Linear):** Target = `ln(price)` ($R^2 \approx 0.623$)

While the $R^2$ results successfully replicated the explanatory power of the reference paper (Guanajuato study), the coefficient analysis revealed three **econometric paradoxes** that must be addressed before model tuning:

| model\_name | feature\_name | coefficient | standard\_error | p\_value | significance\_level |
| :--- | :--- | :--- | :--- | :--- | :--- |
| Model A \(Linear\) | \_\_INTERCEPT\_\_ | -3569918.99 | 368817.35 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_bathrooms | 688079.31 | 50238.23 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_bedrooms | -492347.23 | 65832.46 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_crime\_consolidated | 5756.01 | 399.49 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_crime\_homicide | 60931.28 | 33203.45 | 0.066 | \* Marginally Significant |
| Model A \(Linear\) | feat\_dist\_center | -122.93 | 17.66 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_dist\_park | -244.36 | 44.65 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_dist\_supermarket | 238.07 | 29.54 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_has\_garden | 354276.87 | 101621.16 | 0.0008 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_m2\_constructed | 22112.04 | 282.79 | 0 | \*\*\* Highly Significant |
| Model A \(Linear\) | feat\_parking\_spots | 486444.87 | 28637.46 | 0 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | \_\_INTERCEPT\_\_ | 14.0734 | 0.0365 | 0 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | feat\_bathrooms | 0.1811 | 0.0051 | 0 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | feat\_bedrooms | -0.0272 | 0.0066 | 0.0001 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | feat\_crime\_consolidated | 0.0005 | 0 | 0 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | feat\_crime\_homicide | -0.0121 | 0.0033 | 0.0004 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | feat\_dist\_center | 0 | 0 | 0.7211 | Not Significant \(Likely Noise\) |
| Model B \(Log-Linear\) | feat\_dist\_park | -0 | 0 | 0.1956 | Not Significant \(Likely Noise\) |
| Model B \(Log-Linear\) | feat\_dist\_supermarket | 0 | 0 | 0.0874 | \* Marginally Significant |
| Model B \(Log-Linear\) | feat\_has\_garden | 0.1832 | 0.0102 | 0 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | feat\_m2\_constructed | 0.002 | 0 | 0 | \*\*\* Highly Significant |
| Model B \(Log-Linear\) | feat\_parking\_spots | 0.0393 | 0.0029 | 0 | \*\*\* Highly Significant |


### A. The "Negative Bedroom" Paradox
* **Observation:** The coefficient for `feat_bedrooms` was **negative** (-492,347 MXN in Linear model).
* **Interpretation:** The model suggests that adding a bedroom *reduces* the house value by half a million pesos, holding other factors constant.
* **Diagnosis:** High probability of **Multicollinearity**. Since `feat_m2_constructed` (size) is already in the model, `bedrooms` and `size` are fighting to explain the same variance. A small house with many rooms implies tiny, lower-value rooms.

### B. The "Crime Value" Paradox
* **Observation:** The coefficient for `feat_crime_consolidated` was **positive** (+5,756 MXN).
* **Interpretation:** The model implies that higher crime rates are associated with *higher* property values.
* **Diagnosis:** **Endogeneity / Omitted Variable Bias**. Crime is likely acting as a proxy for **economic activity**. Wealthy areas attract more opportunistic crime than undeveloped areas. The model is confusing "wealthy target" with "valuable location."

### C. The "Vanishing Distance" Effect
* **Observation:** In the Log-Linear Model (Model B), distance variables (Park, Center) became statistically insignificant ($p\text{-value} > 0.05$).
* **Diagnosis:** The logarithmic transformation of the price compressed the variance so much that the subtle linear effect of distance was lost. This suggests the relationship between distance and price might be non-linear or strictly local (e.g., only matters within 500m).

---
## 2. Hypothesis & Action Plan
To resolve these issues and improve $R^2$ > 0.70, we will test the following hypotheses:

1. **H1 (Interaction Features):** Creating interaction terms (e.g., `bathrooms_per_bedroom`) will resolve the multicollinearity between size and room counts.

2. **H2 (Multicollinearity):** Removing `bedrooms` or creating an interaction term (e.g., `bathrooms_per_bedroom`) will stabilize the coefficients of physical characteristics.
3. **H3 (Crime Unbundling):** Instead of a consolidated crime rate, we will separate **Violent Crime** (expected negative impact) from **Property Crime** (proxy for wealth) to isolate the true "fear factor."
4. **H4 (Rich Features):** Incorporating specific amenities (Schools, Hospitals) will capture the variance currently attributed to "noise" in the Baseline.

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from statsmodels.stats.outliers_influence import variance_inflation_factor
from dotenv import load_dotenv

# Visualization Settings
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [12, 8]
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Initialize BigQuery Client
load_dotenv()

try:
    client = bigquery.Client()
    print("Successful connection to BigQuery (Authenticated via Environment Variables)")
except Exception as e:
    print(f"Connection Error: {e}")
    print("Tip: Verify that your .env file has the correct path to the JSON.")

print("Libraries loaded. Ready for diagnostics.")

Successful connection to BigQuery (Authenticated via Environment Variables)
Libraries loaded. Ready for diagnostics.


## 3. Data Loading: Unleashing the Full OBT (One Big Table)

In Phase A (Baseline), we intentionally restricted our analysis to a limited subset of variables to strictly replicate the methodology of the reference paper (Guanajuato study).

**Transition to Phase B:**
Now that the baseline is established, we proceed to unlock the full potential of our **One Big Table (OBT)**. Unlike the original study, our dataset includes a richer set of granular features collected from multiple sources.

**Key Additions in this Phase:**
* **Crime Granularity:** Instead of a consolidated sum, we access specific crime types (Homicide, Burglary, Violence, Vehicle Theft) to test the "Unbundling" hypothesis.
* **Expanded Amenities:** We include previously unused proximity features (Schools, Hospitals, etc.) to capture value drivers that were treated as "noise" in the baseline.
* **Full Physical Specs:** Detailed breakdown of property characteristics (e.g., Half-bathrooms, Age of construction).

We will now query the full `view_training_data_cleaned` to perform the advanced diagnostic.

In [22]:
# Query to fetch all potential features + target
query = """
SELECT
    -- Target
    target_price,

    -- Physical Features
    feat_m2_constructed,
    feat_m2_terrain,
    feat_bedrooms,
    feat_bathrooms,
    feat_parking_spots,

    -- Amenities
    feat_is_new,
    feat_has_security,
    feat_has_garden,
    feat_has_pool,
    feat_has_gym,
    feat_has_kitchen,
    feat_has_terrace,

    -- Municipality
    feat_municipality,

    -- Geographic Features (Distances)
    feat_dist_mall,
    feat_dist_park,
    feat_dist_industrial,
    feat_dist_green_area,
    feat_dist_playground,
    feat_dist_service,
    feat_dist_convenience,
    feat_dist_market,
    feat_dist_supermarket,
    feat_dist_center,
    feat_dist_tourism,

    -- Crime Features (Raw Counts)
    feat_crime_residential,
    feat_crime_vehicle,
    feat_crime_passerby,
    feat_crime_homicide,
    feat_crime_injueries,
    feat_crime_drug_dealing,
    feat_crime_violent

FROM `real-estate-qro.queretaro_data_marts.obt_listings_valuation_features`
WHERE target_price IS NOT NULL
"""

# Load to DataFrame
df = client.query(query).to_dataframe()

# Quick sanity check
print(f"Dataset Shape: {df.shape}")
df.head()

C:\Users\a-b-e\anaconda3\envs\real_estate_ds\lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Dataset Shape: (11840, 32)


,target_price,feat_m2_constructed,feat_m2_terrain,feat_bedrooms,feat_bathrooms,feat_parking_spots,feat_is_new,feat_has_security,feat_has_garden,feat_has_pool,...,feat_dist_supermarket,feat_dist_center,feat_dist_tourism,feat_crime_residential,feat_crime_vehicle,feat_crime_passerby,feat_crime_homicide,feat_crime_injueries,feat_crime_drug_dealing,feat_crime_violent
0,3300000.000,39000.000,39000.000,5.000,5.000,5.000,1,0,0,0,...,16135.467,16371.798,2437.980,26.621,0.000,3.328,9.983,126.452,29.949,0.000
1,4440000.000,290.000,212.000,3.000,3.000,2.000,1,0,1,1,...,20000.000,9662.670,12920.501,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,3000000.000,124.000,90.000,3.000,3.000,1.000,1,0,1,1,...,20000.000,1844.340,20000.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,2690000.000,150.000,115.000,3.000,3.000,2.000,1,1,1,1,...,17730.578,16905.141,8779.006,53.115,5.902,0.000,0.000,132.786,20.656,2.951
4,2990000.000,175.000,115.000,3.000,3.000,2.000,1,1,1,1,...,17730.578,16905.141,8779.006,53.115,5.902,0.000,0.000,132.786,20.656,2.951


## 4. Comprehensive Data Integrity Check
Before proceeding to feature engineering, we must validate the integrity of the raw OBT data loaded from BigQuery. Unlike the initial SQL filtering, this step uses Pandas to detect subtle anomalies that could bias the model.

**We are looking for:**
1.  **Unexpected Zeros:** Are "0 bedrooms" real studio apartments or missing data?
2.  **Missing Values (NaNs):** Quantifying the emptiness of new columns.
3.  **Skewness:** Identifying which variables need the Log-Transformation proposed in **H4**.
4.  **Logic Errors:** e.g., Construction size > Land size (in single-family homes).

In [23]:
# --- DATA INTEGRITY REPORT ---

def check_integrity(df):
    """
    Generates a summary report of the dataframe's health.
    """
    # 1. Basic Info
    print(f"Dataset Shape: {df.shape[0]} rows, {df.shape[1]} columns")

    # 2. Nulls & Zeros Analysis
    integrity_df = pd.DataFrame({
        'Dtype': df.dtypes,
        'Nulls': df.isnull().sum(),
        'Null_Pct': (df.isnull().sum() / len(df)) * 100,
        'Zeros': (df == 0).sum(),
        'Zero_Pct': ((df == 0).sum() / len(df)) * 100,
        'Skewness': df.select_dtypes(include=[np.number]).skew()
    })

    # Filter only relevant columns (exclude IDs or non-numeric if needed)
    return integrity_df.sort_values(by='Null_Pct', ascending=False)

# Execute the check
integrity_report = check_integrity(df)

# Display the report with highlighting for potential issues
# - High Null % -> Candidates for dropping or imputation
# - High Skewness (> 1 or < -1) -> Candidates for Log Transformation (H4)
# - Unexpected Zeros -> Potential data quality issues
print("\n--- Integrity Report (Top 15 Features by Nulls) ---")
display(integrity_report.head(15))

# --- LOGIC CHECKS (Domain Specific) ---
print("\n--- Domain Logic Validation ---")

# Check 1: Houses with 0 Bedrooms or 0 Bathrooms
zero_beds = df[df['feat_bedrooms'] == 0].shape[0]
zero_baths = df[df['feat_bathrooms'] == 0].shape[0]
print(f"Suspicious Records: 0 Bedrooms = {zero_beds}, 0 Bathrooms = {zero_baths}")

# Check 2: Construction > Land (Valid for apartments, suspicious for houses if we had that flag)
# For now, we just check the ratio
denser_than_land = df[df['feat_m2_constructed'] > df['feat_m2_land']].shape[0]
print(f"Properties with Construction > Land Area: {denser_than_land} (Normal for vertical housing)")

# Check 3: Price Outliers (Simple Min/Max check)
print(f"Price Range: ${df['target_price'].min():,.0f} - ${df['target_price'].max():,.0f}")

Dataset Shape: 11840 rows, 32 columns

--- Integrity Report (Top 15 Features by Nulls) ---


,Dtype,Nulls,Null_Pct,Zeros,Zero_Pct,Skewness
feat_bathrooms,float64,0,0.000,363,3.066,1.615
feat_bedrooms,float64,0,0.000,331,2.796,29.497
feat_crime_drug_dealing,float64,0,0.000,2,0.017,-0.680
feat_crime_homicide,float64,0,0.000,94,0.794,-0.901
feat_crime_injueries,float64,0,0.000,2,0.017,-1.899
feat_crime_passerby,float64,0,0.000,67,0.566,-0.096
feat_crime_residential,float64,0,0.000,2,0.017,-0.079
feat_crime_vehicle,float64,0,0.000,3,0.025,-0.759
feat_crime_violent,float64,0,0.000,61,0.515,-1.007
feat_dist_center,float64,0,0.000,42,0.355,0.066



--- Domain Logic Validation ---
Suspicious Records: 0 Bedrooms = 331, 0 Bathrooms = 363


KeyError: 'feat_m2_land'

## 5. Diagnosing Multicollinearity (The "Bedroom" Issue)

To confirm why the bedroom coefficient is negative, we need to check how strongly the physical variables relate to each other.

**Tools used:**
1.  **Correlation Matrix:** Visual inspection of relationships.
2.  **VIF (Variance Inflation Factor):** A statistical measure quantifying how much the variance of a coefficient is inflated due to multicollinearity.
    * *Rule of Thumb:* VIF > 5 indicates high multicollinearity. VIF > 10 requires immediate action (dropping variables).

In [ ]:
# --- A. Correlation Matrix Heatmap ---
# Calculate correlation matrix
corr_matrix = df[['target_price', 'feat_bedrooms', 'feat_bathrooms',
                  'feat_m2_constructed', 'feat_parking_spots']].corr()

# Plot
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool)) # Hide upper triangle

sns.heatmap(corr_matrix,
            mask=mask,
            annot=True,
            fmt=".2f",
            cmap='RdBu_r',
            vmax=1,
            vmin=-1,
            linewidths=.5)

plt.title('Correlation: Physical Features vs Price', fontsize=14)
plt.show()

# --- B. Variance Inflation Factor (VIF) Calculation ---
# Select only numeric features for VIF check (excluding target)
features_for_vif = df[['feat_bedrooms', 'feat_bathrooms',
                       'feat_m2_constructed', 'feat_parking_spots']].dropna()

# Create VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = features_for_vif.columns
vif_data["VIF"] = [variance_inflation_factor(features_for_vif.values, i)
                   for i in range(len(features_for_vif.columns))]

print("\n--- Variance Inflation Factor (VIF) Results ---")
print(vif_data.sort_values(by="VIF", ascending=False))